In [1]:
x=[0.08;0.08;0.08];
jx=j*x; %reactance of line 1-2, 2-3, 3-1
B=diag(imag(2./jx));
B(1,2:3)=-imag(1./jx(2:3));
B(2,[1,3])=-imag(1./jx([1,3]));
B(3,1:2)=-imag(1./jx(1:2));

In [2]:
function Sline=calcLineFlow(V,theta,x)
    sline=zeros(length(x),2);
    % 1->2
    Sline(1,1)=V(1)*V(2)*sin(theta(1)-theta(2))+j*(V(1)^2-V(1)*V(2)*cos(theta(1)-theta(2)))/x(1);
    % 2->1
    Sline(1,2)=V(1)*V(2)*sin(theta(2)-theta(1))+j*(V(2)^2-V(1)*V(2)*cos(theta(2)-theta(1)))/x(1);
    % 2->3
    Sline(2,1)=V(2)*V(3)*sin(theta(2)-theta(3))+j*(V(2)^2-V(2)*V(3)*cos(theta(2)-theta(3)))/x(2);
    % 3->2
    Sline(2,2)=V(3)*V(2)*sin(theta(3)-theta(2))+j*(V(3)^2-V(3)*V(2)*cos(theta(3)-theta(2)))/x(2);
    % 3->1
    Sline(3,1)=V(3)*V(1)*sin(theta(3)-theta(1))+j*(V(3)^2-V(3)*V(1)*cos(theta(3)-theta(1)))/x(3);
    % 1->3
    Sline(3,2)=V(1)*V(3)*sin(theta(1)-theta(3))+j*(V(1)^2-V(1)*V(3)*cos(theta(1)-theta(3)))/x(3);

endfunction

In [5]:
format long
V=ones(3,1);
theta=zeros(3,1);
scaling=1
iP=[2,3]
iQ=[2]
PG=[nan;0.0;1.0]*scaling
QG=[nan;0.0;nan]*scaling
PL=[0.0;1.5;0.0]*scaling
QL=[0.0;0.6;0.0]*scaling
Vs=[1.04;nan;1.02]
V(!isnan(Vs))=Vs(!isnan(Vs))

itermax=20
epsilon_f=1e-5;
epsilon_x=1e-5;

for iter=1:itermax
    Vdot=V.*exp(j*theta);
    S=Vdot.*conj(j*B*Vdot);
    dy=[PG(iP)-PL(iP)-real(S(iP));QG(iQ)-QL(iQ)-imag(S(iQ))];
    J=zeros(3,3);
    % dP2/dtheta2
    J(1,1)=B(2,1)*V(1)*cos(theta(2)-theta(1))+B(2,3)*V(3)*cos(theta(2)-theta(3));
    % dP2/dtheta3
    J(1,2)=-B(2,3)*V(2)*V(3)*cos(theta(2)-theta(3));
    % dP2/dV2
    J(1,3)=B(2,1)*V(1)*sin(theta(2)-theta(1))+B(2,3)*V(3)*sin(theta(2)-theta(3));

    % dP3/dtheta2
    J(2,1)=-B(3,2)*V(3)*V(2)*cos(theta(3)-theta(2));
    % dP3/dtheta3
    J(2,2)=B(3,1)*V(3)*V(1)*cos(theta(3)-theta(1))+B(3,2)*V(3)*V(2)*cos(theta(3)-theta(2));
    % dP3/dV2
    J(2,3)=B(3,2)*V(2)*sin(theta(3)-theta(2));

    % dQ2/dtheta2
    J(3,1)=-B(2,1)*V(2)*V(1)*sin(theta(2)-theta(1))-B(2,3)*V(2)*V(3)*sin(theta(2)-theta(3));
    % dQ2/dtheta3
    J(3,2)=B(2,3)*V(2)*V(3)*sin(theta(2)-theta(3));
    % dQ2/dV2
    J(3,3)=-2*B(2,2)*V(2)-B(2,1)*V(1)*cos(theta(2)-theta(1))-B(2,3)*V(3)*cos(theta(2)-theta(3));
    dx=J\dy;
    theta(iP)=theta(iP)+dx(1:length(iP));
    V(iQ)=V(iQ)+dx((length(iP)+1):end);
    Vdot=V.*exp(j*theta);
    if max(abs(dx))<epsilon_x && max(abs(dy))<epsilon_f
        disp("---------------------------------");
        disp(sprintf("solution found at %d",iter));
        disp("V=");disp(V);disp("theta=");disp(theta);
        disp("");
        Sline=calcLineFlow(V,theta,x);
        disp(sprintf("P from %d -> %d = %g",1,2,real(Sline(1,1))));
        disp(sprintf("P from %d -> %d = %g",2,1,real(Sline(1,2))));
        disp(sprintf("P from %d -> %d = %g",2,3,real(Sline(2,1))));
        disp(sprintf("P from %d -> %d = %g",3,3,real(Sline(2,2))));
        disp(sprintf("P from %d -> %d = %g",3,1,real(Sline(3,1))));
        disp(sprintf("P from %d -> %d = %g",1,3,real(Sline(3,2))));
        disp(sprintf("Q from %d -> %d = %g",1,2,imag(Sline(1,1))));
        disp(sprintf("Q from %d -> %d = %g",2,1,imag(Sline(1,2))));
        disp(sprintf("Q from %d -> %d = %g",2,3,imag(Sline(2,1))));
        disp(sprintf("Q from %d -> %d = %g",3,2,imag(Sline(2,2))));
        disp(sprintf("Q from %d -> %d = %g",3,1,imag(Sline(3,1))));
        disp(sprintf("Q from %d -> %d = %g",1,3,imag(Sline(3,2))));

        break;
    end
end
if iter==itermax
    disp("---------------------------------");
    disp("solution not found");
end

scaling = 1
iP =

   2   3

iQ = 2
PG =

   NaN
     0
     1

QG =

   NaN
     0
   NaN

PL =

                   0
   1.500000000000000
                   0

QL =

                   0
   0.600000000000000
                   0

Vs =

   1.040000000000000
                 NaN
   1.020000000000000

V =

   1.040000000000000
   1.000000000000000
   1.020000000000000

itermax = 20
---------------------------------
solution found at 4
V=
   1.040000000000000
   1.004348001484383
   1.020000000000000
theta=
                       0
  -5.157190269294110e-02
   1.305099985569765e-02

P from 1 -> 2 = 0.0538441
P from 2 -> 1 = -0.0538441
P from 2 -> 3 = -0.0661559
P from 3 -> 3 = 0.0661559
P from 3 -> 1 = 0.0138441
P from 1 -> 3 = -0.0138441
Q from 1 -> 2 = 0.480835
Q from 2 -> 1 = -0.430229
Q from 2 -> 3 = -0.169771
Q from 3 -> 2 = 0.226292
Q from 3 -> 1 = -0.253871
Q from 1 -> 3 = 0.261129
